In [1]:
#Author: Tongshu Zheng from Duke University
#Email: tongshu.zheng@duke.edu; contact me if you have any questions regarding the code
#Please reference the code source and publication (i.e., "Estimating ground-level PM2.5 using micro-satellite 
#images by a convolutional neural network and random forest approach") if you use the code.

In [16]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
# Where to save the figures
PROJECT_ROOT_DIR = "."
PROJECT_SAVE_DIR = "Project"

import os
if not (os.path.isdir(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR)):
    print('Figure directory didn''t exist, creating now.')
    os.mkdir(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR)
else:
    print('Figure directory exists.') 
    
def savepdf(fig,name):
    fig.savefig(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR+'/'+name+'.pdf')

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import math
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from scipy.interpolate import interp1d
from datetime import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import os
import requests
from requests.auth import HTTPBasicAuth
import json
from multiprocessing.dummy import Pool as ThreadPool
import time
import shlex, subprocess
import urllib.request
import webbrowser
from os import listdir
from os.path import isfile, join

Figure directory exists.


In [5]:
#Beijing 35 stations
my_lat = [39.929,
39.886,
39.929,
39.878,
39.982,
39.937,
39.987,
40.09,
40.002,
39.863,
39.824,
39.914,
39.742,
39.718,
39.795,
39.886,
40.127,
40.217,
39.937,
40.143,
40.328,
40.37,
40.453,
40.292,
40.365,
40.499,
40.1,
39.712,
39.52,
39.58,
39.899,
39.876,
39.954,
39.856,
39.939,
         39.955079]
my_long = [116.417,
116.407,
116.339,
116.352,
116.397,
116.461,
116.287,
116.174,
116.207,
116.279,
116.146,
116.184,
116.136,
116.404,
116.506,
116.663,
116.655,
116.23,
116.106,
117.1,
116.628,
116.832,
115.972,
116.22,
115.988,
116.911,
117.12,
116.783,
116.3,
116,
116.395,
116.394,
116.349,
116.368,
116.483,
116.467977]

In [6]:
beijing_locations = ['东四', '天坛', '官园', '万寿西宫', '奥体中心', '农展馆', '万柳', '北部新区', '植物园', '丰台花园',
       '云岗', '古城', '房山', '大兴', '亦庄', '通州', '顺义', '昌平', '门头沟', '平谷', '怀柔', '密云',
       '延庆', '定陵', '八达岭', '密云水库', '东高村', '永乐店', '榆垡', '琉璃河', '前门', '永定门内',
       '西直门北', '南三环', '东四环','US Embassy']

#Shanghai stations
my_lat = [31.238, 
31.2036, 
31.3008, 
31.1654, 
31.2659, 
31.0935, 
31.2261, 
31.1907, 
31.2284, 
31.2071]
my_long = [121.4, 
121.478, 
121.467, 
121.412, 
121.536, 
120.978, 
121.425, 
121.703, 
121.533, 
121.577 ]

#for shanghai
beijing_locations = ['普陀 ',
'十五厂',
'虹口', 
'徐汇上师大', 
'杨浦四漂', 
'青浦淀山湖', 
'静安监测站', 
'浦东川沙', 
'浦东新区监测站', 
'浦东张江' ]

# Type in your info here!

In [ ]:
session = requests.Session()
API_key_1 = '' #type in your planet account API_key here!!
password_1 = '' #type in your planet account password here!!
API_key_2 = '' #it would be great if you have another planet account to deal with errors in image downloading when 
#sometimes the default account may fail; if you do not have a second account, then just let API_key_2 = API_key_1
password_2 = '' #same, if you do not have a second account, then just let password_2 = password_1
session.auth = (API_key_1,password_1)

# Fuctions to retrieve, clip, and download images; no need to change anything

In [4]:
item_type1 = "PSScene3Band"
id_all = []
id_cannot_activate_all = []
clipping_id_list_all = []
download_link_all = []
def handle_page(page, id_list):
    for item in page["features"]:
        print (item["id"])
        id_list.append(item["id"])
def fetch_page(search_url,id_list):
    page = session.get(search_url).json()
    handle_page(page,id_list)
    next_url = page["_links"].get("_next")
    if next_url:
        fetch_page(next_url, id_list)
def activate_item(item_id):
    print ("activating: " + item_id)

    # request an item
    item = session.get(
        ("https://api.planet.com/data/v1/item-types/" +
        "{}/items/{}/assets/").format(item_type1, item_id))

    # request activation
    session.post(
        item.json()["visual"]["_links"]["activate"])

def activation_status_checker(item_id):
    print ("Checking activation status: " + item_id)
    # request an item
    item = session.get(
        ("https://api.planet.com/data/v1/item-types/" +
        "{}/items/{}/assets/").format(item_type1, item_id))
    
    # retrieve status
    try:
        activation_status_result = requests.get(
        item.json()["visual"]["_links"]["_self"],
        auth=HTTPBasicAuth(API_key_1,password_1)    
      )
    except:
        activation_status_result = requests.get(
        item.json()["visual"]["_links"]["_self"],
        auth=HTTPBasicAuth(API_key_2,password_2)
      )
    
    if activation_status_result.json()["status"] == 'active':
        return True
    else:
        return False

def clip_request(item_id):
    args = ['curl',
     '-u',
     API_key_1+':',
     '-X',
     'POST',
     '-H',
     'Content-Type: application/json',
     'https://api.planet.com/compute/ops/clips/v1/',
     '-d',
     '{\n    "aoi": {\n                "type": "Polygon",\n        "coordinates": [\n          [\n            [\n              '+str(first_point[1])+',\n              '+str(first_point[0])+'\n            ],\n            [\n              '+str(second_point[1])+',\n              '+str(second_point[0])+'\n            ],\n            [\n              '+str(third_point[1])+',\n              '+str(third_point[0])+'\n            ],\n            [\n              '+str(fourth_point[1])+',\n              '+str(fourth_point[0])+'\n            ],\n            [\n              '+str(fifth_point[1])+',\n              '+str(fifth_point[0])+'\n            ]\n          ]\n        ]},\n    "targets": [\n      {\n        "item_id": '+json.dumps(str(item_id))+',\n        "item_type":'+json.dumps(item_type1)+',\n        "asset_type": "visual"\n      }\n    ]\n}']
    process = subprocess.Popen(args, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    current_loads = json.loads(stdout)
    clipping_id_list.append(current_loads['id'])
    print ("clipping: " + item_id)
    return clipping_id_list

def clipping_status_checker(item_id, clipping_id):
    print ("Checking clipping status: " + item_id)
    # request an item
    args = ['curl',
     '-u',
     API_key_1+':',
     'https://api.planet.com/compute/ops/clips/v1/'+str(clipping_id)]
    process = subprocess.Popen(args, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    try:
        status_loads = json.loads(stdout)
    except:
        pass
    # request activation
    try:
        if status_loads["state"] == 'succeeded':
            download_links.append(status_loads['_links']['results'][0])
            return True
        else:
            return False
    except:
        return False

parallelism = 2
thread_pool = ThreadPool(parallelism)    

In [3]:
import datetime
time_tracker = dict()

## Downloading images!

# Type in some filter parameters here:

In [ ]:
#The spatial coverage of your final cropped image, default to 1 km
coverage = 1
#Define your desirable cloudiness; cloudiness = 0 means completely clear days; cloudiness = 1
# means all the images regardless of the cloudiness; default to 0.05
cloudiness = 0.05
# your start and end dates in the example format
start_date = '2017-01-01T00:00:00.000Z'
end_date = '2019-07-21T00:00:00.000Z'

In [ ]:
# the geo json geometry object we got from geojson.io
for (i, (long, lat)) in enumerate(zip(my_long,my_lat)):
    print(i)
    if i>=0:
        PROJECT_ROOT_DIR = "."
        PROJECT_SAVE_DIR = beijing_locations[i]
        if not (os.path.isdir(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR+'_2017')):
            print('Image downloading directory didn''t exist, creating now.')
            os.mkdir(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR+'_2017')
        id_list = []
        id_cannot_activate = []
        clipping_id_list = []
        download_links = []
        # Define starting point.
        #lat, long
        start = geopy.Point(lat, long)

        # Define a general distance object, initialized with a distance of ? km.
        d = geopy.distance.VincentyDistance(kilometers = 2**0.5*0.5*coverage)

        # Use the `destination` method with a bearing of 0 degrees (which is north)
        # This defines the 1km * 1km bounding box with the station of interest at the center

        first_point = d.destination(point=start, bearing=315)
        second_point = d.destination(point=start, bearing=45)
        third_point = d.destination(point=start, bearing=135)
        fourth_point = d.destination(point=start, bearing=225)
        fifth_point = d.destination(point=start, bearing=315)

        #long, lat
        geo_json_geometry = {
          "type": "Polygon",
          "coordinates":  [
                  [
                    [
                      first_point[1],
                      first_point[0]
                    ],
                    [
                      second_point[1],
                      second_point[0]
                    ],
                    [
                      third_point[1],
                      third_point[0]
                    ],
                    [
                      fourth_point[1],
                      fourth_point[0]
                    ],
                    [
                      fifth_point[1],
                      fifth_point[0]
                    ]
                  ]
                ]
        }

        # filter for items the overlap with our chosen geometry
        geometry_filter = {
          "type": "GeometryFilter",
          "field_name": "geometry",
          "config": geo_json_geometry
        }

        # filter images acquired in a certain date range
        date_range_filter = {
          "type": "DateRangeFilter",
          "field_name": "acquired",
          "config": {
            #"gte": "2017-10-01T00:00:00.000Z",
            #"lte": "2018-09-30T00:00:00.000Z"
            #  "gte": "2018-10-01T00:00:00.000Z",
            #  "lte": "2019-07-21T00:00:00.000Z"
            #"gte": "2017-01-01T00:00:00.000Z",
            #"lte": "2017-09-30T00:00:00.000Z"
            "gte": start_date,
            "lte": end_date
          }
        }

        # filter any images which are more than 5% clouds
        cloud_cover_filter = {
          "type": "RangeFilter",
          "field_name": "cloud_cover",
          "config": {
            "lte": cloudiness
          }
        }


        # create a filter that combines our geo and date filters
        # could also use an "OrFilter"
        redding_reservoir = {
          "type": "AndFilter",
          "config": [geometry_filter, date_range_filter, cloud_cover_filter]
        }
        
        stats_endpoint_request = {
            "name": 'very_large_search',
          "interval": "day",
          "item_types": [item_type1],
          "filter": redding_reservoir
        }
        #searching qualified images
        result = \
          requests.post(
            'https://api.planet.com/data/v1/searches',
            auth=HTTPBasicAuth(API_key_1,password_1),
            json=stats_endpoint_request)
        
        saved_search_id = result.json()["id"]
        print(saved_search_id)
        
        first_page = \
    ("https://api.planet.com/data/v1/searches/{}" +
        "/results?_page_size={}").format(saved_search_id, 250)
    
        fetch_page(first_page, id_list)
        
        # activating images
        thread_pool.map(activate_item, id_list)
        
        #if i<=-1:
        #    time.sleep(3600)
        #thread_pool.map(activate_item, id_list)
        
        for item_id in id_list:
            start_time = time.time()
            activation_succeeded = False
            while not activation_succeeded:
                if activation_status_checker(item_id):
                    activation_succeeded = True
                    print('Activation of scene '+str(item_id)+ ' succeeded and is ready to download') 
                elif time.time() - start_time > 50:
                    activation_succeeded = True
                    print('Scene '+str(item_id)+ ' cannot be activated')
                    id_list.remove(str(item_id))
                    id_cannot_activate.append(str(item_id))
                else:
                    print("...Still waiting for "+str(item_id)+ " activation to complete...")
                    time.sleep(10)
                    
        #clipping images
        for item_id in id_list:
            try:
                clip_request(item_id)
                time.sleep(0.3)
            except:
                print('Scene '+str(item_id)+ ' does not intersect target AOI')
                id_list.remove(str(item_id))
                id_cannot_activate.append(str(item_id))
                continue
            start_time = time.time()
            clipping_succeeded = False
            
            while not clipping_succeeded:
                if clipping_status_checker(item_id, clipping_id_list[-1]):
                    clipping_succeeded = True
                    print("Clipping of scene succeeded and is ready to download") 
                    time.sleep(1)
                    webbrowser.open(download_links[-1])
                elif time.time() - start_time > 900:
                    clipping_succeeded = True
                    print('Scene '+str(item_id)+ ' cannot be clipped')
                    id_list.remove(str(item_id))
                    clipping_id_list.remove(str(clipping_id_list[-1]))
                    id_cannot_activate.append(str(item_id))
                else:
                    print("...Still waiting for clipping to complete...")
                    time.sleep(5)
                    
            time.sleep(1)
            
        id_all.append(id_list)
        id_cannot_activate_all.append(id_cannot_activate)
        clipping_id_list_all.append(clipping_id_list)
        #download_link_all.append(download_links)
        print(str(beijing_locations[i])+" finished")
        print(datetime.datetime.now())
        time_tracker[beijing_locations[i]]=datetime.datetime.now()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
6d70e05d34fd4bbcbd2220bef0b286fd
20190614_023922_0f52
20190614_023923_0f52
20190612_013753_0f36
20190609_052746_0f02
20190609_052747_0f02
20190602_023809_103c
20190531_023410_0e20
20190529_023739_101b
20190528_023803_1044
20190527_023724_1025
20190527_023723_1025
20190528_024012_102e
20190528_013610_0f4d
20190527_013853_0f21
20190525_013603_1_1054
20190525_013604_1054
20190525_023906_103b
20190523_023632_1024
20190523_023633_1024
20190521_023843_0f17
20190516_023804_1004
20190511_023909_0f4e
20190509_023159_0e26
20190509_023200_0e26
20190505_023943_0f12
20190505_023944_0f12
20190507_023929_101e
20190502_023541_101f
20190502_023542_101f
20190502_014341_1_104a
20190502_014341_104a
20190502_023944_1001
20190502_023945_1001
20190501_023835_1014
20190426_014405_0f46
20190425_023820_1042
20190421_023812_1044
20190426_023759_103c
20190421_023824_1018
20190414_023956_0f28
20190414_023708_0f17
2019

20170428_222725_0c43
20170426_054228_0c42
20170426_054227_0c42
20170425_233109_0c41
20170425_233110_0c41
20170428_230327_0c12
20170426_235122_0c54
20170426_235121_0c54
20170428_222414_0c46
20170428_222413_0c46
20170420_021217_1030
20170423_021833_0e19
20170423_003900_0c81
20170423_021831_0e19
20170423_070548_0d05
20170415_043433_0c44
20170420_021131_1025
20170418_023730_0c78
20170418_023731_0c78
20170418_090250_0c78
20170418_021134_1041
20170417_091909_0c42
20170417_091908_0c42
20170417_021843_0e1f
20170417_021844_0e1f
20170415_021216_1029
20170415_021217_1029
20170415_021831_0e0e
20170415_021830_0e0e
20170413_021823_0e14
20170412_050322_0c27
20170410_021202_100c
20170410_021203_100c
20170409_061721_0c41
20170409_061722_1_0c41
20170409_061637_0c37
20170331_021140_0f28
20170331_021139_0f28
20170302_041232_0c60
20170313_021647_0e26
20170309_015621_1_0c0b
20170309_015621_0c0b
20170302_021803_0e3a
20170224_021710_0e19
20170224_063210_0c37
20170223_065421_0c81
20170212_045125_1_0c78
2017021

activating: 20180410_022802_0f35
activating: 20180427_022811_1009
activating: 20180213_022539_1029
activating: 20180308_022647_1042
activating: 20180605_023020_100e
activating: 20180308_024332_0f36
activating: 20180604_022929_1014
activating: 20180410_023822_0f2a
activating: 20180426_022801_0f31
activating: 20180409_023847_0f40
activating: 20180425_022842_1004
activating: 20180829_053918_0f02
activating: 20180420_022823_1012
activating: 20180829_023218_100e
activating: 20180420_023628_0f3c
activating: 20180212_022541_1035
activating: 20180302_022553_100c
activating: 20180604_022930_1014
activating: 20180824_023215_1032
activating: 20180212_022558_0e2f
activating: 20180302_022554_100c
activating: 20180601_022910_1002
activating: 20180409_022807_0f15
activating: 20180418_023641_104e
activating: 20180212_022559_0e2f
activating: 20180302_024357_0f24
activating: 20180531_022851_1032
activating: 20180408_023835_101c
activating: 20180418_022751_0f22
activating: 20180212_024558_0f2b
activating

activating: 20170906_022056_0e14
activating: 20170425_233110_0c41
activating: 20170415_021830_0e0e
activating: 20170626_021944_0e26
activating: 20170531_021447_103a
activating: 20170825_022114_0e0e
activating: 20170428_230327_0c12
activating: 20170625_021522_1008
activating: 20170825_022113_0e0e
activating: 20170525_021332_1015
activating: 20170620_022129_0e0d
activating: 20170824_021920_1025
activating: 20170424_021152_1035
activating: 20171102_030308_0f33
activating: 20170825_021733_100a
activating: 20170501_021226_1008
activating: 20170620_022130_0e0d
activating: 20170413_021823_0e14
activating: 20170814_021737_0f18
activating: 20170426_235122_0c54
activating: 20170412_050322_0c27
activating: 20170807_021547_101e
activating: 20170426_235121_0c54
activating: 20170211_051708_0d05
activating: 20170410_021202_100c
activating: 20170428_222414_0c46
activating: 20170131_021615_0e30
activating: 20170620_021519_1029
activating: 20170428_222413_0c46
activating: 20170410_021203_100c
activating

Activation of scene 20190414_023708_0f17 succeeded and is ready to download
Checking activation status: 20190412_023623_103d
Activation of scene 20190412_023623_103d succeeded and is ready to download
Checking activation status: 20190411_014627_1050
Activation of scene 20190411_014627_1050 succeeded and is ready to download
Checking activation status: 20190408_014530_104b
Activation of scene 20190408_014530_104b succeeded and is ready to download
Checking activation status: 20190407_053056_0f02
Activation of scene 20190407_053056_0f02 succeeded and is ready to download
Checking activation status: 20190407_053057_0f02
Activation of scene 20190407_053057_0f02 succeeded and is ready to download
Checking activation status: 20190404_023939_1012
Activation of scene 20190404_023939_1012 succeeded and is ready to download
Checking activation status: 20190403_023607_0f35
Activation of scene 20190403_023607_0f35 succeeded and is ready to download
Checking activation status: 20190402_024022_0f12


...

...Still waiting for clipping to complete...
Checking clipping status: 20180113_025016_1020
...Still waiting for clipping to complete...
Checking clipping status: 20180113_025016_1020
Clipping of scene succeeded and is ready to download
clipping: 20180113_022435_0f18
Checking clipping status: 20180113_022435_0f18
...Still waiting for clipping to complete...
Checking clipping status: 20180113_022435_0f18
...Still waiting for clipping to complete...
Checking clipping status: 20180113_022435_0f18
...Still waiting for clipping to complete...
Checking clipping status: 20180113_022435_0f18
...Still waiting for clipping to complete...
Checking clipping status: 20180113_022435_0f18
...Still waiting for clipping to complete...
Checking clipping status: 20180113_022435_0f18
...Still waiting for clipping to complete...
Checking clipping status: 20180113_022435_0f18
...Still waiting for clipping to complete...
Checking clipping status: 20180113_022435_0f18
...Still waiting for clipping to complete

Checking clipping status: 20180105_022332_102f
Clipping of scene succeeded and is ready to download
clipping: 20180101_022441_1039
Checking clipping status: 20180101_022441_1039
...Still waiting for clipping to complete...
Checking clipping status: 20180101_022441_1039
...Still waiting for clipping to complete...
Checking clipping status: 20180101_022441_1039
...Still waiting for clipping to complete...
Checking clipping status: 20180101_022441_1039
...Still waiting for clipping to complete...
Checking clipping status: 20180101_022441_1039
...Still waiting for clipping to complete...
Checking clipping status: 20180101_022441_1039
...Still waiting for clipping to complete...
Checking clipping status: 20180101_022441_1039
...Still waiting for clipping to complete...
Checking clipping status: 20180101_022441_1039
...Still waiting for clipping to complete...
Checking clipping status: 20180101_022441_1039
...Still waiting for clipping to complete...
Checking clipping status: 20180101_022441

...Still waiting for clipping to complete...
Checking clipping status: 20171218_022328_1001
...Still waiting for clipping to complete...
Checking clipping status: 20171218_022328_1001
...Still waiting for clipping to complete...
Checking clipping status: 20171218_022328_1001
...Still waiting for clipping to complete...
Checking clipping status: 20171218_022328_1001
...Still waiting for clipping to complete...
Checking clipping status: 20171218_022328_1001
...Still waiting for clipping to complete...
Checking clipping status: 20171218_022328_1001
...Still waiting for clipping to complete...
Checking clipping status: 20171218_022328_1001
...Still waiting for clipping to complete...
Checking clipping status: 20171218_022328_1001
Clipping of scene succeeded and is ready to download
clipping: 20171218_025631_1050
Checking clipping status: 20171218_025631_1050
...Still waiting for clipping to complete...
Checking clipping status: 20171218_025631_1050
...Still waiting for clipping to complete

...Still waiting for clipping to complete...
Checking clipping status: 20171213_025625_0f21
Clipping of scene succeeded and is ready to download
clipping: 20171212_025559_1051
Checking clipping status: 20171212_025559_1051
...Still waiting for clipping to complete...
Checking clipping status: 20171212_025559_1051
...Still waiting for clipping to complete...
Checking clipping status: 20171212_025559_1051
...Still waiting for clipping to complete...
Checking clipping status: 20171212_025559_1051
...Still waiting for clipping to complete...
Checking clipping status: 20171212_025559_1051
...Still waiting for clipping to complete...
Checking clipping status: 20171212_025559_1051
...Still waiting for clipping to complete...
Checking clipping status: 20171212_025559_1051
...Still waiting for clipping to complete...
Checking clipping status: 20171212_025559_1051
Clipping of scene succeeded and is ready to download
clipping: 20171211_022330_102e
Checking clipping status: 20171211_022330_102e
..

...Still waiting for clipping to complete...
Checking clipping status: 20171205_022210_1002
Clipping of scene succeeded and is ready to download
clipping: 20171204_022227_1008
Checking clipping status: 20171204_022227_1008
...Still waiting for clipping to complete...
Checking clipping status: 20171204_022227_1008
...Still waiting for clipping to complete...
Checking clipping status: 20171204_022227_1008
...Still waiting for clipping to complete...
Checking clipping status: 20171204_022227_1008
...Still waiting for clipping to complete...
Checking clipping status: 20171204_022227_1008
...Still waiting for clipping to complete...
Checking clipping status: 20171204_022227_1008
...Still waiting for clipping to complete...
Checking clipping status: 20171204_022227_1008
...Still waiting for clipping to complete...
Checking clipping status: 20171204_022227_1008
...Still waiting for clipping to complete...
Checking clipping status: 20171204_022227_1008
...Still waiting for clipping to complete